<a href="https://colab.research.google.com/github/MaratNaz12/ML-DL/blob/main/cifar10_class_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import and data preparation

In [13]:
import os 
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import optimizer

In [14]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
data_dir = './data/cifar10'
dataset = ImageFolder(data_dir + '/train',transform = ToTensor())
valid_size = 5000
train_size = len(dataset) - valid_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])
batch_size = 128
train_dataset_load = DataLoader(train_dataset, batch_size, shuffle = True, num_workers = 2, pin_memory = True)

valid_dataset_load = DataLoader(valid_dataset, batch_size * 2, num_workers = 2, pin_memory = True)

Using downloaded and verified file: ./cifar10.tgz


### NN Model

In [7]:
def accuracy(outputs,labels):
  _,preds = torch.max(outputs,dim = 1)
  return torch.tensor(torch.sum(preds == labels).item()/len(preds))
  

In [8]:
class ImageClasificationBase(nn.Module):

  def training_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    return loss

  def validation_step(self,batch):
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(out,labels)
    return {'val_loss':loss.detach(),'val_acc': acc}

  def validation_epoch_end(self,outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.tensor(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.tensor(batch_accs).mean()
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
  
  def epoch_end(self,epoch,result):
     print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['val_loss'], result['val_acc']))
     

In [9]:
class probCNNModel(ImageClasificationBase):
  def __init__(self):
    super().__init__()  
    self.network = nn.Sequential(
        
        nn.Conv2d(3,32, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),# output: 64 x 16 x 16

        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

        nn.Flatten(), 
        nn.Linear(256*4*4, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, xb):
    return self.network(xb)
model = probCNNModel()

### Train model

In [10]:

def evaluate(model,valid_dataset_load):
  model.eval()
  outputs = [model.validation_step(batch) for batch in valid_dataset_load]
  return model.validation_epoch_end(outputs)

def fit(epochs_num, lr, model, train_dataset_load, valid_dataset_load, opt_func = torch.optim.SGD):
  history= []
  train_losses = []
  opt = opt_func(model.parameters(), lr)
  for epoch in range(epochs_num):
    model.train()
    for batch in train_dataset_load:
      loss = model.training_step(batch)
      train_losses.append(loss)
      loss.backward()
      opt.step()
      opt.zero_grad()

    result = evaluate(model,valid_dataset_load)
    result['val_loss'] = torch.tensor(train_losses).mean().item()
    history.append(result)
    model.epoch_end(epoch,result)
  return history



###GPU activvation

In [11]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

train_dataset_load = DeviceDataLoader(train_dataset_load, get_default_device())
valid_dataset_load = DeviceDataLoader(valid_dataset_load, get_default_device())
to_device(model, get_default_device());
model = to_device(probCNNModel(), get_default_device())

### T&T

In [ ]:
epochs_num = 10
opt_func = torch.optim.Adam
lr = 0.001
history = fit(epochs_num, lr, model, train_dataset_load, valid_dataset_load, opt_func)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    val_losses = [x['val_loss'] for x in history]
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('val_loss')
    plt.title('Loss vs. No. of epochs')
plot_losses(history)

In [ ]:
# dataset = ImageFolder(data_dir + '/test',transform = ToTensor())
# def predict_image(img,model):
#   img = to_device(img.unsqueeze(0), get_default_device())
#   pred = model(img)
#   _,pred = torch.max(pred, dim = 1)
#   return dataset.classes[pred[0].item()]
# img,label = dataset[114]
# plt.imshow(img.permute(1,2,0))
# print("Label", dataset.classes[label],'pred', predict_image(img,model))